<a href="https://colab.research.google.com/github/esgantivar/qmc/blob/torch/examples/tn/mnist-qmps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensornetwork -q

In [2]:
# Install qmc if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !pip install git+https://github.com/esgantivar/qmc.git@torch -q
else:
    import sys
    sys.path.insert(0, '../../')

In [3]:
import numpy as np
import tensorflow as tf
import tensornetwork as tn

In [4]:
tn.set_default_backend('tensorflow')

In [5]:
from qmc.tf.tn.models import DMKDClassifierTNSGD

In [6]:
def preprocess_images(x):
  n_data, dim0, dim1 = tuple(x.shape)
  n_sites = dim0 * dim1
  x = x.reshape((n_data, n_sites)) / 255
  sin = tf.sin((np.pi/2) * x)
  cos = tf.cos((np.pi/2) * x)
  return tf.stack([sin, cos],axis=-1)

In [7]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
# convert images to supported format
x_train = preprocess_images(x_train)
x_test = preprocess_images(x_test)

In [18]:
class LayerTN(tf.keras.layers.Layer):
  def __init__(self, n_sites, d_bond, d_phys, **kwargs):
    super(LayerTN, self).__init__(**kwargs)
    self.single_rank = n_sites // 2
    self.n_sites = n_sites
    self.d_bond = d_bond
    self.d_phys = d_phys
    mps = [
        self._boundary(self.d_phys, self.d_bond),
        self._middle(self.d_phys, self.single_rank - 1, self.d_bond),
        self._boundary(self.d_phys, self.d_bond),
        self._middle(self.d_phys, self.single_rank - 1, self.d_bond),
    ]

    self.mps_var = [tf.Variable(node, name=f'mps{i}', trainable=True, dtype=tf.float32) for (i, node) in
                    enumerate(mps)]

  def call(self, inputs, **kwargs):
    def f(input_vec, mps_var):
      l_1n = tn.Node(mps_var[0])
      ln = tn.Node(mps_var[1])
      r_1n = tn.Node(mps_var[2])
      rn = tn.Node(mps_var[3])

      il_1 = tn.Node(input_vec[0])
      il = tn.Node(input_vec[1:self.single_rank])
      ir = tn.Node(input_vec[self.single_rank:self.n_sites - 1])
      ir_1 = tn.Node(input_vec[self.n_sites - 1])

      # Create TN
      il_1[0] ^ l_1n[1]
      l_1n[0] ^ ln[1]
      ln[0] ^ il[0]
      ln[3] ^ il[1]

      ir_1[0] ^ r_1n[1]
      r_1n[0] ^ rn[1]
      rn[0] ^ ir[0]
      rn[3] ^ ir[1]

      ln[2] ^ rn[2]

      # Contract
      ans = tn.contractors.greedy(tn.reachable(ln))
      return ans.tensor

    result = tf.vectorized_map(
        lambda vec: f(vec, self.mps_var),
        inputs
    )
    return result

  @staticmethod
  def _middle(d_phys, n_sites, bond):
    return np.random.normal(0, 1e-2, size=(n_sites, bond, bond, d_phys))

  @staticmethod
  def _boundary(d_phys, bond):
    return np.random.normal(0, 1e-2, size=(bond, d_phys))

In [19]:
class ModelTN(tf.keras.Model):
    def __init__(self, num_classes=10, n_sites=28 ** 2, d_bond=8, d_phys=2, **kwargs):
        super(ModelTN, self).__init__()
        self.d_bond = d_bond
        self.n_sites = n_sites
        self.d_phys = d_phys
        self.num_classes = num_classes
        self.qmd = []
        for _ in range(num_classes):
            self.qmd.append(LayerTN(n_sites=n_sites, d_bond=d_bond, d_phys=self.d_phys))

    def call(self, inputs, **kwargs):
        probs = []
        for i in range(self.num_classes):
            probs.append(self.qmd[i](inputs))
        posteriors = tf.stack(probs, axis=-1)
        return tf.nn.softmax(posteriors)


d_bond = 16

In [20]:
model = ModelTN(n_sites=28 ** 2, d_bond=16, num_classes=10, d_phys=2)

In [21]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              metrics=['accuracy'])

In [22]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 27s 21ms/step - loss: 0.7648 - accuracy: 0.7529 - val_loss: 0.3361 - val_accuracy: 0.9012
Epoch 2/20
469/469 [==============================] - 6s 12ms/step - loss: 14.1605 - accuracy: 0.9118 - val_loss: 52.5084 - val_accuracy: 0.5419
Epoch 3/20
469/469 [==============================] - 6s 12ms/step - loss: 8.4545 - accuracy: 0.8227 - val_loss: 1.0280 - val_accuracy: 0.9209
Epoch 4/20
469/469 [==============================] - 6s 12ms/step - loss: 0.7571 - accuracy: 0.9285 - val_loss: 0.5389 - val_accuracy: 0.9206
Epoch 5/20
469/469 [==============================] - 6s 12ms/step - loss: 0.4156 - accuracy: 0.9359 - val_loss: 0.3180 - val_accuracy: 0.9358
Epoch 6/20
469/469 [==============================] - 6s 12ms/step - loss: 0.2633 - accuracy: 0.9413 - val_loss: 0.2437 - val_accuracy: 0.9432
Epoch 7/20
469/469 [==============================] - 6s 12ms/step - loss: 0.2016 - accuracy: 0.9462 - val_loss: 0.2134 - val_accuracy: 0.9

dbond=22

In [23]:
model = ModelTN(n_sites=28 ** 2, d_bond=22, num_classes=10, d_phys=2)

In [24]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              metrics=['accuracy'])

In [25]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 19s 21ms/step - loss: 7.0194 - accuracy: 0.6359 - val_loss: 0.2772 - val_accuracy: 0.9150
Epoch 2/20
469/469 [==============================] - 6s 13ms/step - loss: 0.2798 - accuracy: 0.9172 - val_loss: 0.2294 - val_accuracy: 0.9317
Epoch 3/20
469/469 [==============================] - 6s 13ms/step - loss: 0.2168 - accuracy: 0.9362 - val_loss: 0.2079 - val_accuracy: 0.9395
Epoch 4/20
469/469 [==============================] - 6s 13ms/step - loss: 0.1869 - accuracy: 0.9435 - val_loss: 0.1844 - val_accuracy: 0.9469
Epoch 5/20
469/469 [==============================] - 6s 13ms/step - loss: 0.1628 - accuracy: 0.9520 - val_loss: 0.1931 - val_accuracy: 0.9418
Epoch 6/20
469/469 [==============================] - 6s 13ms/step - loss: 0.1590 - accuracy: 0.9535 - val_loss: 0.1977 - val_accuracy: 0.9428
Epoch 7/20
469/469 [==============================] - 6s 13ms/step - loss: 0.1468 - accuracy: 0.9561 - val_loss: 0.1763 - val_accuracy: 0.950